## 2.0 Google Analytics Customer Revenue Prediction

### Importing packages

In [2]:
# File system manangement
import os
# Suppress warnings
import warnings
warnings.filterwarnings('ignore')
# Numpy and pandas for data manipulation
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', 300)
# Importing random for random selections
import random
# Json for importing JSON columns
import json as json
# Pandas io json normalizing
from pandas.io.json import json_normalize
# Scipy stats for statistical analysis
import scipy.stats as stats
# sklearn preprocessing for dealing with categorical features
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
# to use for woe binning for features with a large number of categories
import scorecardpy as sc
# Light gradient boost classifier
from lightgbm import LGBMRegressor
# Sklearn Inputing data spliting method
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
# Sklearn importing auc as measurement metric
from sklearn.metrics import mean_squared_error
# Gc memory managment
import gc
# Matplotlib pyplot for plotting
import matplotlib.pyplot as plt
# Seabourne for visualization
import seaborn as sns
sns.set_style('white')

### Importing data

In [ ]:
# Setting up package to import data converting JSON columns into individual features

# https://www.kaggle.com/julian3833/1-quick-start-read-csv-and-flatten-json-fields
JSON_COLUMNS = ['device', 'geoNetwork', 'totals', 'trafficSource']
def load_df(csv_path):
    df = pd.read_csv(csv_path, converters={column: json.loads for column in JSON_COLUMNS}\
                     , dtype={'fullVisitorId': 'str', 'visitStartTime': 'str', 'date': 'str'})
    for column in JSON_COLUMNS:
        column_as_df = json_normalize(df[column])
        column_as_df.columns = [f"{column}.{subcolumn}" for subcolumn in column_as_df.columns]
        df = df.drop(column, axis=1).merge(column_as_df, right_index=True, left_index=True)
    return df

In [ ]:
# Seeting the input and output directory
os.chdir('C:/Users/Jake Cherrie/Documents/Projects/Gstore-Revenue-Prediction')
# Viewing the contained datasets
os.listdir('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction')

In [ ]:
# Importing the training data
trn_df = load_df('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/train.csv')
trn_df['totals.transactionRevenue'].fillna(0, inplace = True)

In [ ]:
# Quick view of training data
trn_df.info()

In [ ]:
# Droping features with no information
drp_cols = [col for col in trn_df.columns if trn_df[col].nunique() == 1 & trn_df[col].notnull().values.all()]
trn_df = trn_df.drop(columns=drp_cols)
# Dropping duplicate feature visitStartTime is the same as visitId
trn_df = trn_df.drop(columns='visitId')
# Dropping capaign code as there is ony 1 non-null entry
trn_df = trn_df.drop(columns='trafficSource.campaignCode')
# Noticed that for the training set channelGrouping contains all the information in trafficSource.medium so droping trafficSource.medium
trn_df = trn_df.drop(columns='trafficSource.medium')

In [ ]:
# Saving memory by shortning ints and floats
def size_reduction(df):
    int_col     = df.select_dtypes(include=[np.int64]).columns
    flt_col     = df.select_dtypes(include=[np.float64]).columns 
    df[int_col] = df[int_col].astype(np.int32)
    df[flt_col] = df[flt_col].astype(np.float32)
    
size_reduction(trn_df)
gc.collect()

In [ ]:
trn_df.describe(include='all')

In [ ]:
# Importing the testing data
tst_df = load_df('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/test.csv')

In [ ]:
# Quick view of training data
tst_df.info()

In [ ]:
# Droping features with no information
drp_cols = [col for col in tst_df.columns if tst_df[col].nunique() == 1 & tst_df[col].notnull().values.all()]
tst_df = tst_df.drop(columns=drp_cols)
# Dropping duplicate feature visitStartTime is the same as visitId
tst_df = tst_df.drop(columns='visitId')
tst_df = tst_df.drop(columns='trafficSource.medium')

size_reduction(tst_df)
gc.collect()

In [ ]:
tst_df.describe(include='all')

### Saving Data for Quick Loads and Debugging

In [ ]:
trn_df.to_csv('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/trn.csv', index=False)
tst_df.to_csv('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/tst.csv', index=False)

In [ ]:
# Seting up debugging sets for quicker exploration
debugTrn = sorted(random.sample(range(1,903653),903653-200000))
debugTst = sorted(random.sample(range(1,804684),804684-100000))

In [1]:
trn_df = pd.read_csv('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/trn.csv'\
           , dtype={'fullVisitorId': 'str', 'visitStartTime': 'str', 'date': 'str'}, skiprows=0)
tst_df = pd.read_csv('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/tst.csv'\
           , dtype={'fullVisitorId': 'str', 'visitStartTime': 'str', 'date': 'str'}, skiprows=0)
gc.collect()

NameError: name 'pd' is not defined

### Creating New Combination Features

In [ ]:
# Setting up a has revenue flag
trn_df['hasRevenue'] = (trn_df['totals.transactionRevenue'] > 0).astype(int)
trn_df['logRevenue'] = np.log1p(trn_df['totals.transactionRevenue'])

### Creating Time Features

In [ ]:
#Looking at first and last shows that it is clearly a time dependent problem
trn_df['visitStartTime'] = pd.to_datetime(trn_df['visitStartTime'],unit='s')
print(trn_df['visitStartTime'].describe())
tst_df['visitStartTime'] = pd.to_datetime(tst_df['visitStartTime'],unit='s')
print(tst_df['visitStartTime'].describe())
# should explore weighting the more ecent data lgb(weight=w)
trn_df['weight'] = 1

In [ ]:
trn_df['weekday'] = trn_df['visitStartTime'].dt.weekday.astype(str)
tst_df['weekday'] = tst_df['visitStartTime'].dt.weekday.astype(str)

In [ ]:
trn_df['time']  = trn_df['visitStartTime'].dt.hour.astype(str)
tst_df['time']  = tst_df['visitStartTime'].dt.hour.astype(str)

### Encoding Categorical Features

#### Weekday

In [ ]:
# Plotting the weekday relationship to the log1p revenue
sns.barplot('weekday', 'logRevenue', data = trn_df)

In [ ]:
# Ploting count per group
sns.countplot('weekday', data=trn_df)

In [ ]:
# Weight of Evidence (WOE) encoding
weekday = trn_df[['weekday','hasRevenue']]
#bins_weekday = sc.woebin(weekday, y='hasRevenue', stop_limit=0.02, max_num_bin=2, method='tree')
#Saving for quick load    
#np.save('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/Bins sets/bins_weekday.npy', bins_weekday) 
# Quick load of dataframe
bins_weekday = np.load('C:/Users/Jake Cherrie/Data Sets/Gstore Revenue Prediction/Bins sets/bins_weekday.npy').item()
bins_weekday

In [ ]:
# Applying WOE encoding
trn_df['weekday'] = sc.woebin_ply(trn_df, bins_weekday)['weekday_woe']
tst_df['weekday'] = sc.woebin_ply(tst_df, bins_weekday)['weekday_woe']

In [ ]:
#Dropping unused features
trn_set = trn_df.drop(columns=['fullVisitorId', 'visitStartTime', 'date', 'sessionId'\
                                , 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.keyword'])
tst_set = tst_df.drop(columns=['fullVisitorId', 'visitStartTime', 'date', 'sessionId'\
                                , 'trafficSource.adwordsClickInfo.gclId', 'trafficSource.keyword'])

In [ ]:
trn_fts = trn_set[tst_set.columns]
trn_tgt = trn_set['logRevenue']
tst_fts = tst_set

In [ ]:
#Setting up categorical columns
cat_col = [col for col in trn_fts.columns if trn_fts[col].dtype == 'object']
# factorizing categorical columns
for col in cat_col:
    indexer = pd.factorize(trn_set[col])[1]
    trn_fts[col] = indexer.get_indexer(trn_fts[col])
    tst_fts[col] = indexer.get_indexer(tst_fts[col])

### Visit ID Level Fit

In [ ]:
# Initializing lgb paramaters
params={'num_leaves': 31,
        'max_depth': -1,
        'learning_rate': 0.05,
        'n_estimators': 100,
        'metric':'rmse',
        'num_leaves': 31,
        'verbose': 1,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "random_state":42,
        'lambda_l2': 0.02085548700474218,
        'lambda_l1': 0.004107624022751344,
        'bagging_fraction': 0.7934712636944741,
        'feature_fraction': 0.686612409641711,
        'min_child_samples': 20
       }

In [ ]:
# Setting up k fold for CV
flds = KFold(n_splits = 5, shuffle=True, random_state = 42)

# Initializing Variables
trn_prd = np.zeros(trn_fts.shape[0])
tst_prd = np.zeros(tst_fts.shape[0])
imp = pd.DataFrame()
MSE = 0

for n_fld, (trn_idx, vld_idx) in enumerate(flds.split(trn_fts, trn_tgt)):
    trn_X, trn_y = trn_fts.iloc[trn_idx], trn_tgt.iloc[trn_idx]
    vld_X, vld_y = trn_fts.iloc[vld_idx], trn_tgt.iloc[vld_idx]
    
    lgb = LGBMRegressor(**params)
    
    # Fit the model
    lgb.fit(trn_X, trn_y)
    
    # applying the model to the validation data
    val_prd = lgb.predict(vld_X)
    val_prd[val_prd < 0] = 0
    # Calculating and outputting the RMSE
    fld_MSE = mean_squared_error(vld_y, val_prd)
    print('Fold %2d RMSE : %.6f' % (n_fld + 1, np.sqrt(fld_MSE)))
    
    # Summing mean squared errors
    MSE += fld_MSE/5
    
    # Applying predictions to the train set weighted by the MSE
    prd = lgb.predict(trn_fts)
    prd[prd < 0] = 0
    trn_prd += prd/np.sqrt(fld_MSE)
    
    # Applying predictions to the test set weighted by the MSE
    prd = lgb.predict(tst_fts)
    prd[prd < 0] = 0
    tst_prd += prd/np.sqrt(fld_MSE)
    
    # Calculating the fold importance
    imp_df = pd.DataFrame()
    imp_df['fts'] = trn_X.columns
    imp_df['imp'] = lgb.booster_.feature_importance(importance_type='gain')
    
    # Summing the fold importances
    imp_df['fld'] = n_fld+1
    imp = pd.concat([imp, imp_df], axis=0, sort=False)

RMSE = np.sqrt(MSE)
print('RMSE : %.6f' % RMSE)

# Scaling the predictions
trn_fts['prd'] = np.expm1(trn_prd*RMSE)
trn_fts['log_prd'] = trn_prd*RMSE
tst_fts['prd'] = np.expm1(tst_prd*RMSE)
tst_fts['log_prd'] = tst_prd*RMSE
#1.678

In [ ]:
# Ploting feature importances
cols = imp[["fts", "imp"]].groupby("fts").mean().sort_values(by="imp", ascending=False)[:40].index
imp['log1p_imp'] = np.log1p(imp['imp'])
best_features = imp.loc[imp.fts.isin(cols)]
plt.figure(figsize=(8, 10))
sns.barplot(x="log1p_imp", y="fts", data=best_features.sort_values(by="imp", ascending=False))
plt.title('Features (avg over folds)')

In [ ]:
# Features: Perform aggregations
aggregations = {
    'channelGrouping': ['median'],
    'visitNumber': ['max', 'mean', 'sum', 'min', 'std'],
    'device.browser': ['median'],
    'device.deviceCategory': ['median'], 
    'device.isMobile': ['max', 'mean', 'sum', 'min'], 
    'device.operatingSystem': ['max', 'mean', 'sum', 'min'],
    'geoNetwork.city': ['median'], 
    'geoNetwork.continent': ['median'], 
    'geoNetwork.country': ['median'],
    'geoNetwork.metro': ['median'], 
    'geoNetwork.networkDomain': ['median'], 
    'geoNetwork.region': ['median'],
    'geoNetwork.subContinent': ['median'], 
    'totals.bounces': ['max', 'mean', 'sum', 'min', 'std'], 
    'totals.hits': ['max', 'mean', 'sum', 'min', 'std'],
    'totals.newVisits': ['max', 'mean', 'sum', 'min', 'std'], 
    'totals.pageviews': ['max', 'mean', 'sum', 'min', 'std'], 
    'trafficSource.adContent': ['median'],
    'trafficSource.adwordsClickInfo.adNetworkType': ['median'],
    #'trafficSource.adwordsClickInfo.gclId': ['mode'],
    'trafficSource.adwordsClickInfo.isVideoAd': ['median'],
    'trafficSource.adwordsClickInfo.page': ['median'],
    'trafficSource.adwordsClickInfo.slot': ['median'], 
    'trafficSource.campaign': ['median'],
    'trafficSource.isTrueDirect': ['median'], 
    #'trafficSource.keyword': ['median'],
    'trafficSource.referralPath': ['median'], 
    'trafficSource.source': ['median'], 
    'weekday': ['median'],
    'time': ['median'],
    'prd': ['max', 'mean', 'sum', 'min'],
    'log_prd': ['max', 'mean', 'sum', 'min']
}

In [ ]:
# Adding in fullVisitor ID for aggregation
trn_fts['fullVisitorId'] = trn_df.fullVisitorId
tst_fts['fullVisitorId'] = tst_df.fullVisitorId

In [ ]:
trn_agg = trn_fts.groupby('fullVisitorId').agg(aggregations)
trn_agg.columns = pd.Index([e[0] + "." + e[1].upper() for e in trn_agg.columns.tolist()])
tst_agg = tst_fts.groupby('fullVisitorId').agg(aggregations)
tst_agg.columns = pd.Index([e[0] + "." + e[1].upper() for e in tst_agg.columns.tolist()])

In [ ]:
tgt_agg = np.log1p(trn_df.groupby('fullVisitorId').sum()['totals.transactionRevenue'])

In [ ]:
trn_agg.shape, tst_agg.shape, tgt_agg.shape

In [ ]:
# Initializing stage 2 paramaters
params={'num_leaves': 31,
        'max_depth': -1,
        'learning_rate': 0.05,
        'n_estimators': 100,
        'metric':'rmse',
        'num_leaves': 31,
        'verbose': 1,
        "subsample": 0.9,
        "colsample_bytree": 0.9,
        "random_state":42,
        'lambda_l2': 0.02085548700474218,
        'lambda_l1': 0.004107624022751344,
        'bagging_fraction': 0.7934712636944741,
        'feature_fraction': 0.686612409641711,
        'min_child_samples': 21
       }

In [ ]:
# Setting up k fold for CV
flds = KFold(n_splits = 5, shuffle=True, random_state = 42)

# Initializing Variables
sub_prd = np.zeros(tst_agg.shape[0])
imp = pd.DataFrame()
MSE = 0

for n_fld, (trn_idx, vld_idx) in enumerate(flds.split(trn_agg, tgt_agg)):
    trn_X, trn_y = trn_agg.iloc[trn_idx], tgt_agg.iloc[trn_idx]
    vld_X, vld_y = trn_agg.iloc[vld_idx], tgt_agg.iloc[vld_idx]
    
    lgb = LGBMRegressor(**params)
    
    # Fit the model
    lgb.fit(trn_X, trn_y)
    
    # applying the model to the validation data
    val_prd = lgb.predict(vld_X)
    val_prd[val_prd < 0] = 0
    # Calculating and outputting the RMSE
    fld_MSE = mean_squared_error(vld_y, val_prd)
    print('Fold %2d RMSE : %.6f' % (n_fld + 1, np.sqrt(fld_MSE)))
    
    # Summing mean squared errors
    MSE += fld_MSE/5
    
    # Applying predictions to the test set weighted by the MSE
    prd = lgb.predict(tst_agg)
    prd[prd < 0] = 0
    sub_prd += prd/np.sqrt(fld_MSE)
    
    # Calculating the fold importance
    imp_df = pd.DataFrame()
    imp_df['fts'] = trn_X.columns
    imp_df['imp'] = lgb.booster_.feature_importance(importance_type='gain')
    
    # Summing the fold importances
    imp_df['fld'] = n_fld+1
    imp = pd.concat([imp, imp_df], axis=0, sort=False)

RMSE = np.sqrt(MSE)
print('RMSE : %.6f' % RMSE)

In [ ]:
tst_agg['predictedLogRevenue'] = sub_prd
tst_agg['predictedLogRevenue'].to_csv('submission.csv', header = True, index = True)